# Shopping Cart Demo

This demo shows how to use Webtask to automate adding items to a shopping cart with multiple sequential tasks.

## Setup

In [1]:
from webtask import Webtask
from webtask.integrations.llm import Gemini
import os

# Initialize
llm = Gemini(model="gemini-2.5-flash", api_key=os.getenv("GOOGLE_API_KEY"))
wt = Webtask(headless=False)

# Create stateful agent (remembers context across tasks)
agent = await wt.create_agent(llm=llm, stateful=True)

/Users/stevewang/Github/web-auto/webtask/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Task 1: Add Flat-Head Wood Screws

In [2]:
result1 = await agent.do(
    "Go to https://practicesoftwaretesting.com/ and add 2 Flat-Head Wood Screws to the cart"
)

print(f"Status: {result1.status}")
print(f"Feedback: {result1.feedback}")

Status: Status.COMPLETED
Feedback: Added 2 Flat-Head Wood Screws to the cart.


## Task 2: Add Cross-Head Screws

In [3]:
result2 = await agent.do("Add 5 Cross-head screws to the cart")

print(f"Status: {result2.status}")
print(f"Feedback: {result2.feedback}")

Status: Status.COMPLETED
Feedback: Added 5 Cross-head screws to the cart.


## Task 3: Verify Cart with Structured Output

In [4]:
from pydantic import BaseModel, Field
from typing import List

class CartItem(BaseModel):
    """A single item in the shopping cart."""
    name: str = Field(description="Product name")
    quantity: int = Field(description="Quantity of this item")

class CartVerification(BaseModel):
    """Shopping cart contents."""
    items: List[CartItem] = Field(description="List of items in the cart")
    total_items: int = Field(description="Total number of items in cart")

# Agent remembers previous tasks, so it knows what to verify
result3 = await agent.do(
    "Go to the cart page and extract all items with their quantities",
    output_schema=CartVerification
)

print(f"Status: {result3.status}")
print(f"Feedback: {result3.feedback}")

# result3.output is already a CartVerification instance
if result3.output:
    print(f"\nTotal Items: {result3.output.total_items}")
    print("\nCart Items:")
    for item in result3.output.items:
        print(f"  - {item.name}: {item.quantity}")

Status: Status.COMPLETED
Feedback: Extracted all items and their quantities from the cart page.

Total Items: 7

Cart Items:
  - Flat-Head Wood Screws: 2
  - Cross-head screws: 5


## Cleanup

In [5]:
await wt.close()